# CH03.2.1. **Advanced CNN - Residual block**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [1]:
SEED_NUM = 2025
BATCH_SIZE = 512
EPOCH_NUM = 500
USE_CHECKPOINT_YN = 'Y'
MODEL_PTH = '../../model/cifaClfResidualBlock.pt'

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [2]:
#(1) Import libraries
import os
import random
import tqdm
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import sklearn
import torch
import torchvision
import torchinfo

#(2) Set options
os.environ['PYTHONHASHSEED'] = str(SEED_NUM)
random.seed(a=SEED_NUM)
np.random.seed(seed=SEED_NUM)
torch.use_deterministic_algorithms(mode=True)
torch.manual_seed(seed=SEED_NUM)
torch.mps.manual_seed(seed=SEED_NUM)

#(3) Define device(hardware)
if torch.backends.mps.is_available() :
    device = torch.device(device='mps')
else :
    device = torch.device(device='cpu')
print(f'>> Device : {device}')

>> Device : mps


#### 00.2. **사용자정의함수 정의**

In [3]:
#(1) Define `show_img()` function
def show_img(df:torchvision.datasets, index:int) :
    img = df[index][0]
    target = df[index][1]
    img = (img/2+0.5).numpy()
    channel_cnt = img.shape[0]
    if channel_cnt == 3 :
        img = np.transpose(a=img, axes=(1, 2, 0))
        plt.imshow(X=img) 
    elif channel_cnt == 1 : 
        img = np.squeeze(a=img, axis=0)
        plt.imshow(X=img, cmap='gray')
    else : 
        pass 
    plt.xlabel(xlabel=f'Target : {target}({df.classes[target]})')
    plt.show()

#(2) Define `compute_metrics()` function
def compute_metrics(model:torch.nn.Module, loader:torch.utils.data.DataLoader) -> pd.DataFrame :
    preds = []
    targets = []
    model.eval()
    with torch.no_grad() : 
        for inputs, _ in loader :
            preds.extend(
                torch.argmax(input=model(x=inputs), dim=1).to(device='cpu').numpy()
            )
            targets.extend(
                _.to(device='cpu').numpy()
            )
    model.train()
    accuracy = sklearn.metrics.accuracy_score(y_true=targets, y_pred=preds)
    precision = sklearn.metrics.precision_score(y_true=targets, y_pred=preds, average='weighted')
    recall = sklearn.metrics.recall_score(y_true=targets, y_pred=preds, average='weighted')
    f1 = sklearn.metrics.f1_score(y_true=targets, y_pred=preds, average='weighted')
    output = pd.DataFrame(data={
        'metricName' : ['accuracy', 'precision', 'recall', 'f1'], 
        'value'      : [accuracy, precision, recall, f1] 
    })
    return output

#### 00.3. **클래스 정의**

In [4]:
#(1) Define `ConvResidualBlock` class
class ConvResidualBlock(torch.nn.Module) :
    def __init__(self, in_channels:int, out_channels:int, stride:int) :
        super().__init__()
        self.conv_block_01 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            torch.nn.BatchNorm2d(num_features=out_channels),
            torch.nn.ReLU()

        )
        self.conv_block_02 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            torch.nn.BatchNorm2d(num_features=out_channels)
        )
        
        CON = (in_channels != out_channels) or (stride != 1)
        if CON : 
            self.ds_block = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(num_features=out_channels)
            )
        else :
            self.ds_block = torch.nn.Sequential()
    def forward(self, x:torch.Tensor) -> torch.Tensor :
        identity = x
        x = self.conv_block_01(x)
        x = self.conv_block_02(x)
        identity = self.ds_block(identity)
        x += identity
        x = torch.nn.functional.relu(x)
        return x

#(2) Define `MyClassificationModel` class
class MyClassificationModel(torch.nn.Module) :
    def __init__(self, input_shape:list, channel_dims:list, device:torch.device) :
        super().__init__()
        self.device = device
        self.conv_block = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_shape[0], out_channels=channel_dims[1], kernel_size=3, stride=1, padding=1, bias=False),
            torch.nn.BatchNorm2d(num_features=channel_dims[1]),
            torch.nn.ReLU()
        )
        self.res_blocks = torch.nn.ModuleList()
        res_block_num = len(channel_dims)
        for i in range(res_block_num-2) :
            if i == 0 :
                stride = 1
            else :
                stride = 2
            res_block = ConvResidualBlock(in_channels=channel_dims[i+1], out_channels=channel_dims[i+2], stride=stride)
            self.res_blocks.append(res_block)
        self.global_pool = torch.nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc_block = torch.nn.Sequential(
            torch.nn.Linear(in_features=channel_dims[-1], out_features=10),
            torch.nn.LogSoftmax(dim=1)
        )
        self.to(device=device)
    def forward(self, x: torch.Tensor) -> torch.Tensor :
        x = x.to(self.device)
        x = self.conv_block(x)
        for layer in self.res_blocks :
            x = layer(x)
        x = self.global_pool(x)
        x = x.reshape(shape=(x.shape[0], -1))
        x = self.fc_block(x)
        return x

<b></b>

## 01. **데이터셋 전처리 및 로드**

#### 01.1. **이미지 전처리 파이프라인 정의**

In [5]:
img_tf = torchvision.transforms.Compose(
    transforms=[
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]
)

#### 01.2. **데이터셋 로드 및 전처리 적용**

In [6]:
cifa_train = torchvision.datasets.CIFAR10(root='../../data', train=True, download=True, transform=img_tf)
cifa_test = torchvision.datasets.CIFAR10(root='../../data', train=False, download=True, transform=img_tf)

#### 01.3. **EDA**

In [7]:
#(1) Print sample of train
# len(cifa_train)

In [8]:
#(2) Display image
# show_img(df=cifa_train, index=1)

In [9]:
#(3) Check `input_size`
input_shape = list(cifa_train[0][0].shape)

#(4) Print `input_size`
# input_shape

In [10]:
#(5) Print frequency of target class
# target_freq = collections.Counter()
# for i in range(len(cifa_train)):
#     input, target = cifa_train[i]
#     if isinstance(target, torch.Tensor) :
#         target = target.item()
#     target_freq[target] += 1
# pd.DataFrame(data=list(target_freq.items()), columns=['class', 'count']).sort_values(by='class')

#### 01.4. **데이터로더 변환**

In [11]:
cifa_train_loader = torch.utils.data.DataLoader(dataset=cifa_train, batch_size=BATCH_SIZE, shuffle=True)
cifa_test_loader = torch.utils.data.DataLoader(dataset=cifa_test, batch_size=BATCH_SIZE, shuffle=True)

<b></b>

## 02. **모델 구축 및 학습**

#### 02.1. **하이퍼 파라미터 정의**

In [12]:
channel_dims = [16, 32, 16, 8]

#### 02.2. **모델 정의**

In [13]:
#(1) Define `model`
model = MyClassificationModel(input_shape=input_shape, channel_dims=channel_dims, device=device).to(dtype=torch.float32)

#(2) Check `model`'s parameter number
torchinfo.summary(
    model=model, 
    input_size=[BATCH_SIZE]+input_shape,
    device=device
)

Layer (type:depth-idx)                   Output Shape              Param #
MyClassificationModel                    [512, 10]                 --
├─Sequential: 1-1                        [512, 32, 32, 32]         --
│    └─Conv2d: 2-1                       [512, 32, 32, 32]         864
│    └─BatchNorm2d: 2-2                  [512, 32, 32, 32]         64
│    └─ReLU: 2-3                         [512, 32, 32, 32]         --
├─ModuleList: 1-2                        --                        --
│    └─ConvResidualBlock: 2-4            [512, 16, 32, 32]         --
│    │    └─Sequential: 3-1              [512, 16, 32, 32]         4,640
│    │    └─Sequential: 3-2              [512, 16, 32, 32]         2,336
│    │    └─Sequential: 3-3              [512, 16, 32, 32]         544
│    └─ConvResidualBlock: 2-5            [512, 8, 16, 16]          --
│    │    └─Sequential: 3-4              [512, 8, 16, 16]          1,168
│    │    └─Sequential: 3-5              [512, 8, 16, 16]          592
│  

In [14]:
#(3) Define loss function
criterion = torch.nn.NLLLoss()

#(4) Define optimizer(optimization method)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, weight_decay=1e-7)

#(5) Define Scheduler
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=30, gamma=0.1)

#### 02.3. **모델 체크포인트 로드**

In [15]:
init_epoch = 0 
train_cost_hist = []
best_train_cost = float('inf')
if USE_CHECKPOINT_YN == 'Y' :
    try :
        checkpoint = torch.load(f=MODEL_PTH, map_location=device)
        model.load_state_dict(state_dict=checkpoint['model'])
        optimizer.load_state_dict(state_dict=checkpoint['optimizer'])
        init_epoch = checkpoint['best_epoch']
        train_cost_hist = checkpoint['train_cost_hist']
        best_train_cost = train_cost_hist[-1]
    except Exception :
        pass
print(f'>> Last Epoch={init_epoch}, Last Train Loss={best_train_cost}')

>> Last Epoch=245, Last Train Loss=0.6281971055634168


#### 02.4. **모델 학습**

In [16]:
batch_len = len(cifa_train_loader)
best_epoch = init_epoch
progress_bar = tqdm.trange(init_epoch+1, EPOCH_NUM+1)
for epoch in progress_bar : 
    train_cost = 0.0
    model.train()
    for inputs, targets in cifa_train_loader :
        optimizer.zero_grad() 
        preds = model(x=inputs)
        loss = criterion(input=preds, target=targets.to(device=device))
        loss.backward()
        optimizer.step()
        train_cost += loss.item()
    train_cost = train_cost / batch_len
    train_cost_hist.append(train_cost)
    if train_cost < best_train_cost :
        best_epoch = epoch
        best_train_cost = train_cost
        torch.save(
            obj={
                'model'           : model.state_dict(),
                'optimizer'       : optimizer.state_dict(),
                'best_epoch'      : best_epoch,
                'train_cost_hist' : train_cost_hist,
            }, 
            f=MODEL_PTH
        )
    # scheduler.step()
    progress_bar.set_postfix(ordered_dict={'last_epoch':epoch, 'last_train_loss':train_cost, 'best_epoch':best_epoch, 'best_train_cost':best_train_cost})

100%|██████████| 255/255 [51:50<00:00, 12.20s/it, last_epoch=500, last_train_loss=0.56, best_epoch=497, best_train_cost=0.558] 


<b></b>

## 03. **모델 평가**

#### 03.1. **최적 성능 모델 로드**

In [17]:
checkpoint = torch.load(f=MODEL_PTH, map_location=device)
model.load_state_dict(state_dict=checkpoint['model'])
print(f'>> Best Epoch : {np.argmin(a=checkpoint["train_cost_hist"])+1}, Best Train Loss : {np.min(a=checkpoint["train_cost_hist"])}')

>> Best Epoch : 497, Best Train Loss : 0.5581551072548847


#### 03.2. **과소 적합 확인**

In [18]:
#(1) Plot training loss
plt.figure(figsize=(12, 6))
plt.xlabel(xlabel='epoch')
plt.ylabel(ylabel='loss')
plt.plot(train_cost_hist, label='Training Loss')
plt.axvline(x=np.argmin(a=checkpoint["train_cost_hist"]), color='grey', linestyle='--', linewidth=0.6, label=f'Best Training Loss')
plt.legend(loc='upper right')
plt.show()

In [19]:
#(2) Check metrics
compute_metrics(model=model, loader=cifa_train_loader)

,metricName,value
0,accuracy,0.781760
1,precision,0.792724
2,recall,0.781760
3,f1,0.779701


#### 03.3. **일반화 성능 평가**

In [20]:
compute_metrics(model=model, loader=cifa_test_loader)

,metricName,value
0,accuracy,0.665100
1,precision,0.675049
2,recall,0.665100
3,f1,0.663312
